In [2]:
import cv2
import numpy as np
import pickle

cap = cv2.VideoCapture(0)

prototxt = "models/deploy.prototxt"
model = "models/res10_300x300_ssd_iter_140000.caffemodel"

net = cv2.dnn.readNetFromCaffe(prototxt, model)

with open("FaceModel.pkl", 'rb') as model_file:
    clf = pickle.load(model_file)

hog = cv2.HOGDescriptor()

names={
    1: "David",
    2: "Random"
}

while True:

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    h, w, _ = frame.shape

    frame_resize = cv2.resize(frame, (300, 300))

    blob = cv2.dnn.blobFromImage(frame_resize, 1.0, (300,300), (104, 117, 123))

    net.setInput(blob)
    detections = net.forward()

    box = np.array([0,0,0,0], dtype=np.float64)
    area = 0

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    for detection in detections[0][0]:
        if detection[2] > 0.6:
            box_ = detection[3:7] * [w, h, w, h]

            if ((box_[2] - box_[0]) * (box_[3] - box_[1])) > area:
                area = (box_[2] - box_[0]) * (box_[3] - box_[1])
                box = box_

    if area > 0 and len(detections) > 0:

        roi = gray[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
        roi = cv2.resize(roi, (64,128))

        hog_features = hog.compute(roi).flatten().reshape(1, -1)

        prediction = clf.predict(hog_features)

        cv2.imshow("ROI", roi)

        cv2.rectangle(frame, 
                      (int(box[0]), int(box[1])), 
                      (int(box[2]), int(box[3])), 
                      (255,0,0), 2)
        
        cv2.putText(frame,
                    f'Person: {names[int(prediction[0])]}', (10,30),
                    1,1, (255,0,0), 2)

    
    k = cv2.waitKey(10)

    if k == ord('q'):
        break

    cv2.imshow("Frame", frame)

cap.release()
cv2.destroyAllWindows()